# Chapter 4

## Query Enhancement

In [ ]:
%load_ext autoreload
%autoreload 2

import asyncio
import json
import pathlib

import cohere
import weave

import wandb

In [ ]:
WANDB_ENTITY = "rag-course"
WANDB_PROJECT = "dev"

wandb.require("core")

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    group="Chapter 4",
)


weave_client = weave.init(f"{WANDB_ENTITY}/{WANDB_PROJECT}")

In [ ]:
# Reload the data from Chapter 3
chunked_artifact = run.use_artifact(
    f"{WANDB_ENTITY}/{WANDB_PROJECT}/chunked_data:latest", type="dataset"
)
artifact_dir = chunked_artifact.download()
chunked_data_file = pathlib.Path(f"{artifact_dir}/documents.jsonl")
chunked_data = list(map(json.loads, chunked_data_file.read_text().splitlines()))
chunked_data[:2]

In [ ]:
# The query enhancer will be used to identify the language, the intent of the query and generate search queries for retrieval
# this additional information will be used to inform the response generator and improve the retrieval process


In [ ]:
from scripts.query_enhancer import QueryEnhancer
from scripts.utils import display_source

query_enhancer = QueryEnhancer()
display_source(query_enhancer)

In [ ]:
response = await query_enhancer.predict("How do I log images in lightning with wandb?")

In [ ]:
# the retriever remains the same

from scripts.retriever import BM25Retriever
retriever = BM25Retriever()
retriever.index_data(chunked_data)

In [ ]:
# we need to improve the Response generator to incorporate the new information from the query enhancer
# such as adding in the language identification and the intent action to the prompt

from scripts.response_generator import QueryEnhanedResponseGenerator
display_source(QueryEnhanedResponseGenerator)


In [ ]:
from scripts.rag_pipeline import QueryEnhancedRAGPipeline
display_source(QueryEnhancedRAGPipeline)

In [ ]:
# lets add the new prompt
QUERY_ENHANCED_PROMPT = open("prompts/query_enhanced_system.txt").read()

response_generator = QueryEnhanedResponseGenerator(
    model="command-r-plus", prompt=QUERY_ENHANCED_PROMPT, client=cohere.AsyncClient()
)

In [ ]:
# similarily we will also improve the rag pipeline to first call the query enhancer and the use the information
# to include logic to aviod retrieval when the query is unrelated, needs more info, etc.
# extract the language and intent actions so we can instruct the response generator to use them

rag_pipeline = QueryEnhancedRAGPipeline(
    query_enhancer=query_enhancer,
    retriever=retriever,
    response_generator=response_generator,
)

response = await rag_pipeline.predict("How do I log images in lightning with wandb?")
response

## Evaluate and Compare

In [ ]:
eval_dataset = weave.ref(
    "weave:///rag-course/dev/object/Dataset:9O0EmmPINmYjgbXW3kucVrDxlTUQJQs0fVZYJj2mtOk"
).get()

In [ ]:
from scripts.response_metrics import ALL_METRICS as RESPONSE_METRICS

response_evaluations = weave.Evaluation(
    name="Response_Evaluation",
    dataset=eval_dataset,
    scorers=RESPONSE_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"]},
)
query_enhanced_response_scores = asyncio.run(
    response_evaluations.evaluate(rag_pipeline)
)

Exercise 